## Simple tests for grabbing frames

OpenCV has a nice wrapper around webcams. Let's test it there to get the simplest code that we would need in our library.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
np.set_printoptions(precision=2, suppress=True)
# define plots to be inserted interactively
import matplotlib.pyplot as plt
%matplotlib inline
import time
import cv2
cv2.__version__

/usr/local/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


'3.1.0'

In [6]:
class Source:
    def __init__(self, w=640, h=480):
        self.h, self.w = h, w
        import cv2
        self.cap = cv2.VideoCapture(0)
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, self.w)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.h)
      
    def grab(self):
        # grab a frame
        returned, cam_data = self.cap.read()
        data = cam_data.reshape((self.h, self.w, 3))
        return data
    
    def close(self):
        self.cap.release()

start = time.time()
cam = Source()
cam.close()
print('Time to start and stop the camera = ',  time.time() - start , '(s)')


NameError: name 'time' is not defined

This is implemented in the ``LeCheapEyeTracker``class:

In [7]:
from LeCheapEyeTracker import LeCheapEyeTracker

cam = LeCheapEyeTracker()
cam.print_info()
cam.close()


ImportError: No module named 'LeCheapEyeTracker'

In [8]:
from LeCheapEyeTracker import LeCheapEyeTracker
for DOWNSCALE in [1, 2, 4, 8, 16]:
    start = time.time()
    cam = LeCheapEyeTracker(DOWNSCALE=DOWNSCALE)
    cam.close()
    print('DOWNSCALE = ', DOWNSCALE, 'startup= ',  time.time() - start , '(s)')

ImportError: No module named 'LeCheapEyeTracker'

In [9]:
start = time.time()
cam = LeCheapEyeTracker(DOWNSCALE=2)
img = cam.grab()
img = cam.grab()
cam.close()
print('Time to start, grab 2 frames and stop = ',  time.time() - start , '(s)')
print(img.shape, img.min(), img.max())
plt.imshow(img)

NameError: name 'time' is not defined

In [10]:
from LeCheapEyeTracker import LeCheapEyeTracker
for DOWNSCALE in [1, 2, 4, 8, 16]:
    cam = LeCheapEyeTracker(DOWNSCALE=DOWNSCALE)
    start = time.time()
    img = cam.grab()
    img = cam.grab()
    cam.close()
    print('DOWNSCALE = ', DOWNSCALE, 'grab time= ',  time.time() - start , '(s)')

ImportError: No module named 'LeCheapEyeTracker'

## how fast do we grab frames?

In [11]:
N = 100
ctime = np.zeros(N)
start = time.time()
cam = LeCheapEyeTracker()
for i in range(N):
    img = cam.grab()
    ctime[i] = time.time() - start
cam.close()

NameError: name 'time' is not defined

In [12]:
plt.plot(ctime[1:], np.diff(ctime)*1000, '+')
plt.xlabel('time (s)')
plt.ylabel('inter-frame interval(ms)')
_ = plt.axis('tight')
print ('FPS : ',  N/(ctime[-1]-ctime[0]))

NameError: name 'plt' is not defined

## closing the camera

In [13]:
cam.cap.isOpened()

NameError: name 'cam' is not defined

In [14]:
cam.cap.release()

NameError: name 'cam' is not defined

## threaded mode

Following the documentation @ http://docs.opencv.org/modules/highgui/doc/reading_and_writing_images_and_video.html?highlight=imread it is easy to run parallel captures :

In [15]:
from multiprocessing.pool import ThreadPool
from collections import deque
import cv2
import time

class StatValue:
    def __init__(self, smooth_coef = 0.5):
        self.value = None
        self.smooth_coef = smooth_coef
    def update(self, v):
        if self.value is None:
            self.value = v
        else:
            c = self.smooth_coef
            self.value = c * self.value + (1.0-c) * v

def clock():
    return cv2.getTickCount() / cv2.getTickFrequency()

def draw_str(dst, target, s):
    x, y = target
    cv2.putText(dst, s, (x+1, y+1), cv2.FONT_HERSHEY_PLAIN, 1.0, (0, 0, 0), thickness = 2, lineType=cv2.LINE_AA)
    cv2.putText(dst, s, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.0, (255, 255, 255), lineType=cv2.LINE_AA)


class ThreadSource:
    def __init__(self, w=640, h=480, threads=True):
        self.h, self.w = h, w
        import cv2
        self.cap = cv2.VideoCapture(0)
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, self.w)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.h)

        if threads:
            self.threadn = cv2.getNumberOfCPUs()
        else:
            self.threadn = 1
        self.pool = ThreadPool(processes = self.threadn)
        self.pending = deque()

        self.latency = StatValue()
        self.frame_interval = StatValue()
        self.last_frame_time = clock()        
        self.display = False
        self.ctime = []
        self.N = 0

    def process_frame(self, frame, t0):
        # some intensive computation...
        frame = cv2.medianBlur(frame, 19)
        time.sleep(.1)
        frame = cv2.medianBlur(frame, 19)
        return frame, t0

    def run(self, T=10):
        start = clock()
        while clock()-start <10.:
            while len(self.pending) > 0 and self.pending[0].ready():
                res, t0 = self.pending.popleft().get()
                self.latency.update(clock() - t0)
                if self.display:
                    draw_str(res, (20, 40), "latency        :  %.1f ms" % (self.latency.value*1000))
                    draw_str(res, (20, 60), "frame interval :  %.1f ms" % (self.frame_interval.value*1000))
                    cv2.imshow('Webcam video', res)
                self.ctime.append(time.time() - start)
                self.N += 1
            if len(self.pending) < self.threadn:
                ret, frame = self.cap.read()
                t = clock()
                self.frame_interval.update(t - self.last_frame_time)
                self.last_frame_time = t
                task = self.pool.apply_async(self.process_frame, (frame.copy(), t))
                self.pending.append(task)
            ch = 0xFF & cv2.waitKey(1)
            if ch == 27:
                self.close()
    
    def close(self):
        self.cap.release()
        if self.display: cv2.destroyAllWindows()

start = time.time()
cam = ThreadSource()
ctime = cam.run()
cam.close()
start_nothreads = time.time()
cam_nothreads = ThreadSource(threads=False)
ctime = cam_nothreads.run()
cam_nothreads.close()

ImportError: No module named 'cv2'

In [ ]:
plt.plot(np.array(cam.ctime)[1:]-cam.ctime[0], np.diff(np.array(cam.ctime))*1000, 'b+')
plt.plot(np.array(cam_nothreads.ctime)[1:]-cam_nothreads.ctime[0], np.diff(np.array(cam_nothreads.ctime))*1000, 'rx')
plt.xlabel('time (s)')
plt.ylabel('inter-frame interval(ms)')
_ = plt.axis('tight')
print ('FPS threaded: ',  cam.N/(cam.ctime[-1]-cam.ctime[0]))
print ('FPS one thread: ',  cam_nothreads.N/(cam_nothreads.ctime[-1]-cam_nothreads.ctime[0]))

In practice, we will see nice improvments when the image processing pipeline is more consequent. This threaded mode is enabled by default.

##  creating a video for debugging

It may be sometimes useful to just read out some frames for debugging.

In [ ]:
import os
N_frame = 512 # how much you want?
folder = '/tmp/debug' # where we do save the frames
try: 
    os.mkdir(folder)
except: pass

from LeCheapEyeTracker import LeCheapEyeTracker
cam = LeCheapEyeTracker()
start = cam.clock()
for i in range(N_frame):
    ret, frame = cam.cap.read()
    t = cam.clock() - start   
    # https://docs.python.org/3.3/library/string.html#format-examples
    timestr = '{t:03.5f}'.format(t=t).replace('.', '_')
    fname = 'frame_{i}_at_{t}.png'.format(i=i, t=timestr)
    print(fname)
    cv2.imwrite(os.path.join(folder, fname), frame)

cam.close()